In [1]:
import sys
import os
sys.path.insert(0, os.path.join(os.getcwd(), '../core'))
import kaggle_support as kgs
import importlib
import matplotlib.pyplot as plt
import numpy as np
import cupy as cp
from dataclasses import dataclass, field, fields
import pack_cuda
import pack_vis
import pack_cost
import copy
import time
import pack_ga
from IPython.display import HTML, display, clear_output
pack_cuda.USE_FLOAT32 = True
pack_cuda._ensure_initialized()


local
stop final relax at some point
init CUDA
Detected GPU compute capability: 8.9 (arch=sm_89)
GPU max threads per block: 1024
=== Compiling kernel variant: crystal ===
Defines: ENABLE_CRYSTAL_AXES, ENABLE_OVERLAP_AREA, ENABLE_SEPARATION
Command: /usr/local/cuda/bin/nvcc -O3 -use_fast_math --extra-device-vectorization --ptxas-options=-v,--warn-on-spills -arch=sm_89 -DENABLE_CRYSTAL_AXES -DENABLE_OVERLAP_AREA -DENABLE_SEPARATION -cubin /mnt/d//packing/temp/pack_cuda_saved.cu -o /mnt/d//packing/temp/pack_cuda_crystal.cubin
ptxas info    : 0 bytes gmem, 1172 bytes cmem[3]
ptxas info    : Compiling entry function 'multi_boundary_distance_list_total' for 'sm_89'
ptxas info    : Function properties for multi_boundary_distance_list_total
    0 bytes stack frame, 0 bytes spill stores, 0 bytes spill loads
ptxas info    : Used 36 registers, 404 bytes cmem[0], 16 bytes cmem[2]
ptxas info    : Compiling entry function 'multi_boundary_list_total' for 'sm_89'
ptxas info    : Function properties fo

In [ ]:
import pack_runner
fastMode = False
ga = pack_ga.GA()

ga.N_trees_to_do = np.array([40])
ga.n_generations = 10
ga.population_size = 100
ga.selection_size = [1,2,3,4,5,10,20,30,50]
ga.do_legalize = False

kgs.profiling=False
ga.run()
print(ga.populations[-1].fitness)
print(ga.best_cost_per_generation)

/mnt/d/packing/code/analysis/../core/pack_dynamics.py:61: VisibleDeprecationWarning: This function is deprecated and will be removed in a future release. Use the cupy.from_dlpack() array constructor instead.
  x0 = from_dlpack(x0.toDlpack())
/mnt/d/packing/code/analysis/../core/pack_dynamics.py:93: VisibleDeprecationWarning: This function is deprecated and will be removed in a future release. Use the cupy.from_dlpack() array constructor instead.
  return from_dlpack(tmp_cost[:N].toDlpack()), from_dlpack(res.toDlpack())


Generation 0, Trees 40, Best cost: 0.19561130, Est: 0.48902826, h: 4.391835
Generation 1, Trees 40, Best cost: 0.17793527, Est: 0.44483819, h: 4.191494
[0.17793527 0.1797324  0.18040356 0.18169464 0.18207127 0.18421637
 0.18448666 0.18796194 0.1902201 ]
[[0.1956113 ]
 [0.17793527]]


In [ ]:
import pack_ga2
importlib.reload(pack_ga2)

runner = pack_ga2.Orchestrator(n_generations=10)
runner.ga = pack_ga2.GASinglePopulationOld(N_trees_to_do=40)

runner.ga.population_size = 100
runner.ga.selection_size = [1,2,3,4,5,10,20,30,50]

runner.run()


stop final relax at some point


/mnt/d/packing/code/analysis/../core/pack_dynamics.py:61: VisibleDeprecationWarning: This function is deprecated and will be removed in a future release. Use the cupy.from_dlpack() array constructor instead.
  x0 = from_dlpack(x0.toDlpack())
/mnt/d/packing/code/analysis/../core/pack_dynamics.py:93: VisibleDeprecationWarning: This function is deprecated and will be removed in a future release. Use the cupy.from_dlpack() array constructor instead.
  return from_dlpack(tmp_cost[:N].toDlpack()), from_dlpack(res.toDlpack())


In [58]:
print(runner.ga.population.fitness)
print(np.array(runner.ga.best_cost_per_generation))

[0.17793527 0.1797324  0.18040356 0.18169464 0.18207127 0.18421637
 0.18448666 0.18796194 0.1902201 ]
[0.1956113  0.17793527]
